# GFW Data-API Playground

When developing, to test newly saved code:

- restart kernel
- after each save in the command line use: `!python setup.py develop ` or `!pip install -e .` to install
- then re-import `LMIPy`
- use `py.test` in cli to execute tests

### To release

1. Increment the version in `setup.py`.
2. Push changes to github
3. Checkout to master, pull
3. Remove the `dist` folder
3. In the CLI run `make pypi`
4. Check version on [Colab](https://colab.research.google.com/drive/1tRye2QAytWzuki3DQyg4G7fsFFKlZNic#scrollTo=EN4iMVRm6N0Y)



In [25]:
!python setup.py develop 

from IPython.display import clear_output,display
clear_output()

import LMIPy as lmi
from LMIPy import utils

print(f'LMI ver. {lmi.__version__} ready!')

LMI ver. 0.5.0 ready!


In [ ]:
# {1:2}[3]

# RW User Class

In [26]:
ENV = 'production'
# ENV = 'preproduction'
# ENV = 'staging'

u = lmi.User(env=ENV)

In [27]:
email = 'adam.pain@vizzuality.com'
u.login(email=email)

Password: ········


In [28]:
print(f"user_id: {u._id}")
print(f"role: {u.role}")
print(f"apps: {u.apps}")

TOKEN = u.token

user_id: 59db4eace9c1380001d6e4c3
role: ADMIN
apps: ['rw', 'gfw', 'prep', 'aqueduct', 'forest-atlas', 'data4sdgs', 'gfw-climate', 'gfw-pro', 'ghg-gdp', 'ng']


# Data API

### Get Data-API Keys

In [6]:
ENV = 'production'
# ENV = 'preproduction'
# ENV = 'staging'

In [7]:
## Get data-api creds (staging only; not currently enforced in production)
## Always use Prod RW API TOKEN
creds = lmi.Auth(env=ENV, rw_api_token=TOKEN)

In [8]:
## See keys that I have generated
creds.keys

[GFW Data API Credential: 'AJ Prod Test Key' (does not expire),
 GFW Data API Credential: 'GFW Flagship Production Key' (does not expire),
 GFW Data API Credential: 'GFW Flagship Staging Key' (does not expire),
 GFW Data API Credential: 'AJ data-api' (does not expire)]

In [9]:
## Set data-api key
DATA_API_KEY = creds.keys[1].api_key

### Search Data Catalogue

In [39]:
## Search the catalogue to populate datasets
col = lmi.DataCatalogue(env=ENV, search='umd', token=DATA_API_KEY)
datasets = col.datasets
datasets

{1: GFW Data API Dataset: 'umd_regional_primary_forest_2001',
 2: GFW Data API Dataset: 'umd_glad_landsat_alerts',
 3: GFW Data API Dataset: 'umd_tree_cover_gain',
 4: GFW Data API Dataset: 'umd_modis_burned_areas',
 5: GFW Data API Dataset: 'umd_tree_cover_height_2019',
 6: GFW Data API Dataset: 'umd_soy_planted_area',
 7: GFW Data API Dataset: 'umd_tree_cover_loss',
 8: GFW Data API Dataset: 'umd_tree_cover_density_2010',
 9: GFW Data API Dataset: 'umd_area_2013',
 10: GFW Data API Dataset: 'umd_tree_cover_density_2000',
 11: GFW Data API Dataset: 'umd_glad_sentinel2_alerts_coverage',
 12: GFW Data API Dataset: 'umd_drivers',
 13: GFW Data API Dataset: 'umd_glad_sentinel2_alerts'}

In [40]:
## Select Dataset and populate with metadata as needed
d = datasets[7]
d

GFW Data API Dataset: 'umd_tree_cover_loss'

In [41]:
## Populate metadata etc
d.get_metadata(verbose=True)
d.get_versions(True)
d.get_fields(True)

d.metadata['function']

https://data-api.globalforestwatch.org/dataset/umd_tree_cover_loss/v1.8
https://data-api.globalforestwatch.org/dataset/umd_tree_cover_loss
https://data-api.globalforestwatch.org/dataset/umd_tree_cover_loss/v1.8/fields


'Identifies areas of gross tree cover loss.'

In [43]:
d.metadata

{'added_date': '2014-02-01',
 'analysis': None,
 'cautions': 'In this data set, “tree cover” is defined as all vegetation greater than 5 meters in height, and may take the form of natural forests or plantations across a range of canopy densities. “Loss” indicates the removal or mortality of tree cover and can be due to a variety of factors, including mechanical harvesting, fire, disease, or storm damage. As such, “loss” does not equate to deforestation.\nDue to variation in research methodology and date of content, tree cover, loss, and gain data sets cannot be compared accurately against each other. Accordingly, “net” loss cannot be calculated by subtracting figures for tree cover gain from tree cover loss, and current (post-2000) tree cover cannot be determined by subtracting figures for annual tree cover loss from year 2000 tree cover.\nThe 2011-2019 data was produced using [updated methodology](http://earthenginepartners.appspot.com/science-2013-global-forest/download_v1.7.html). C

### Use Slug Directly

In [44]:
## Instantiate new dataset object using the table name slug directly
dataset_name = 'gfw_forest_carbon_net_flux'

d = lmi.GFWDataset(env=ENV, slug=dataset_name, token=DATA_API_KEY)

In [45]:
## Populate metadata etc
d.get_metadata(verbose=True)
d.get_versions(True)
d.get_fields(True)

d.versions

https://data-api.globalforestwatch.org/dataset/gfw_forest_carbon_net_flux/v20210331
https://data-api.globalforestwatch.org/dataset/gfw_forest_carbon_net_flux
https://data-api.globalforestwatch.org/dataset/gfw_forest_carbon_net_flux/v20210331/fields


['v20210127', 'v20210209', 'v20210305', 'v20210331']

In [46]:
# Queryable fields
d.fields

['is__inpe_prodes',
 'gfw_oil_gas__gfw_fid',
 'inpe_amazonia_prodes__year',
 'rspo_southeast_asia_land_cover_2010__land_cover_class',
 'gfw_wood_fiber__gfw_fid',
 'rspo_oil_palm__rspocert',
 'ibge_bra_biomes__name',
 'gfw_logging__gfw_fid',
 'wdpa_protected_areas__iucn_cat',
 'gfw_forest_carbon_gross_emissions__Mg_CO2e_ha',
 'gfw_forest_carbon_net_flux__Mg_CO2e_ha',
 'gfw_forest_carbon_gross_removals__Mg_CO2e_ha',
 'idn_forest_area__class_compressed',
 'umd_tree_cover_loss__threshold',
 'is__umd_tree_cover_gain',
 'umd_tree_cover_density_2010__threshold',
 'inpe_prodes__gfw_fid',
 'gfw_peatlands__Mg_CO2_ha',
 'gfw_deadwood_carbon__Mg_CO2_ha',
 'gmw_global_mangrove_extent_2016__gfw_fid',
 'gfw_belowground_carbon__Mg_CO2_ha',
 'gfw_aboveground_carbon__Mg_CO2_ha',
 'gfw_litter_carbon__Mg_CO2_ha',
 'landmark_icls__gfw_fid',
 'birdlife_key_biodiversity_areas__gfw_fid',
 'idn_forest_area__gfw_fid',
 'gfw_forest_carbon_net_flux__Mg_CO2e_px',
 'gfw_forest_carbon_gross_emissions__Mg_CO2e_px',
 

In [16]:
## Query using a geostore to do zonal stats
gid = 'c3833748f6815d31bad47d47f147c0f0'

In [24]:
sql = """SELECT SUM(gfw_forest_carbon_net_flux__Mg_CO2e),
              SUM(gfw_forest_carbon_gross_emissions__Mg_CO2e),
              SUM(gfw_forest_carbon_gross_removals__Mg_CO2e)
        FROM data
        WHERE umd_tree_cover_density_2000__threshold >= 30
        OR is__umd_tree_cover_gain = 'true'"""

result = d.query(sql=sql, geostore_id=gid, verbose=True)
result

https://data-api.globalforestwatch.org/dataset/wur_radd_alerts/v20210704/query/json?sql=SELECT+SUM%28gfw_forest_carbon_net_flux__Mg_CO2e%29%2C%0A++++++++++++++SUM%28gfw_forest_carbon_gross_emissions__Mg_CO2e%29%2C%0A++++++++++++++SUM%28gfw_forest_carbon_gross_removals__Mg_CO2e%29%0A++++++++FROM+data%0A++++++++WHERE+umd_tree_cover_density_2010__threshold+%3E%3D+30%0A++++++++OR+is__umd_tree_cover_gain+%3D+%27true%27&geostore_id=c3833748f6815d31bad47d47f147c0f0&geostore_origin=rw


[{'gfw_forest_carbon_gross_emissions__Mg_CO2e': 621932500.51561,
  'gfw_forest_carbon_gross_removals__Mg_CO2e': 209671083.90540004,
  'gfw_forest_carbon_net_flux__Mg_CO2e': 412261415.38779}]